In [ ]:
%matplotlib inline
import numpy as np
from numpy.random import *
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [ ]:
class NN:
    def __init__(self, num_input, num_hidden, num_output, learning_rate):
        self.num_input = num_input
        self.num_hidden = num_hidden
        self.num_output = num_output
        self.learning_rate = learning_rate

        self.w_input2hidden = np.random.random((self.num_hidden, self.num_input))
        self.w_hidden2output = np.random.random((self.num_output, self.num_hidden))
        self.b_input2hidden = np.ones((self.num_hidden))
        self.b_hidden2output = np.ones((self.num_output))

    ##活性化関数（シグモイド関数）
    def activate_func(self, x):
        return 1/(1+np.exp(-x))
    ##活性化関数の微分
    def dactivate_func(self,x):
        return self.activate_func(x)*(1-self.activate_func(x))
    ##ソフトマックス関数
    def softmax_func(self,x):
        C = x.max()
        f = np.exp(x-C)/np.exp(x-C).sum()
        return f
    ##順伝播計算_
    def forward_propagation(self, x):
        u_hidden = np.dot(self.w_input2hidden, x) + self.b_input2hidden
        z_hidden = self.activate_func(u_hidden)
        u_output = np.dot(self.w_hidden2output, z_hidden) + self.b_hidden2output
        z_output = self.softmax_func(u_output)
        return u_hidden, u_output, z_hidden, z_output
    ##逆伝播でδを求める
    def backward_propagation(self,t,u_hidden,z_output):
        t_vec = np.zeros(len(z_output))
        t_vec[t] = 1
        delta_output = z_output - t_vec
        delta_hidden = np.dot(delta_output, self.w_hidden2output * self.dactivate_func(u_hidden))
        return delta_hidden, delta_output
    ##損失関数のパラメータwに関する勾配
    def calc_gradient(self,delta,z):
        dW = np.zeros((len(delta), len(z)))
        for i in range(len(delta)):
            for j in range(len(z)):
                dW[i][j] = delta[i] * z[j]
        return dW
    ##重みをアップデートする
    def update_weight(self,w0,gradE):
        return w0 - self.learning_rate*gradE

In [ ]:
def train(nn, iteration,savefig=False):
    epoch = 0
    for epoch in range(iteration+1):
        grad_i2h = 0
        grad_h2o = 0
        gradbias_i2h = 0
        gradbias_h2o = 0
        n = 0
        err = 0
        rand = randint(0,len(data),100)
        for r in rand:
            u_hidden, u_output, z_hidden, z_output = nn.forward_propagation(data[r])
            delta_hidden, delta_output = nn.backward_propagation(target[r], u_hidden, z_output)
            grad_i2h += nn.calc_gradient(delta_hidden, data[r])
            grad_h2o += nn.calc_gradient(delta_output, z_hidden)
            gradbias_i2h += delta_hidden
            gradbias_h2o += delta_output
        nn.w_input2hidden = nn.update_weight(nn.w_input2hidden, grad_i2h / len(rand))
        nn.w_hidden2output = nn.update_weight(nn.w_hidden2output, grad_h2o / len(rand))
        nn.b_input2hidden = nn.update_weight(nn.b_input2hidden, gradbias_i2h / len(rand))
        nn.b_hidden2output = nn.update_weight(nn.b_hidden2output, gradbias_h2o / len(rand))
        #print grad_h2o
        if epoch%10 == 0:
            print(epoch,",",end="")
            if savefig:
                plt.figure(figsize=(5,5))
                #色の用意
                base_color = ["red","blue","green","yellow","cyan",
                              "pink","brown","gray","purple","orange"]
                colors = [base_color[label] for label in target]
                # 教師データのプロット
                plt.scatter(data[:,0],data[:,1],color=colors,alpha=0.5)

                print("plotting...")
                xx = np.linspace(-15,15,80)
                yy = np.linspace(-15,15,80)
                for xi in xx:
                    for yi in yy:
                        _,_,_,z_output = nn.forward_propagation((xi, yi))
                        cls = np.argmax(z_output) #softmaxのスコアの最大のインデックス
                        score = np.max(z_output)
                        plt.plot(xi, yi, base_color[cls],marker="x",alpha=score)
                    print(".",end="")
                plt.xlim(-15,15)
                plt.ylim(-15,15)
                plt.savefig("{}.png".format(epoch))
                plt.clf()
                print("finish plotting")
            


In [ ]:
#データの用意
num_cls = 5
data,target = make_blobs(n_samples=1000, n_features=2, centers=num_cls)
#色の用意
base_color = ["red","blue","green","yellow","cyan",
              "pink","brown","gray","purple","orange"]
colors = [base_color[label] for label in target]
# 教師データのプロット
plt.scatter(data[:,0],data[:,1],color=colors,alpha=0.5)
plt.savefig("original.png")
# Neural Netの用意
nn = NN(num_input=2,num_hidden=20,num_output=num_cls,learning_rate=0.2)
# 学習
train(nn, iteration=300, savefig=False)

plt.figure(figsize=(5,5))

print("plotting...")
xx = np.linspace(-15,15,70)
yy = np.linspace(-15,15,70)
for xi in xx:
    for yi in yy:
        _,_,_,z_output = nn.forward_propagation((xi, yi))
        cls = np.argmax(z_output) #softmaxのスコアの最大のインデックス
        score = np.max(z_output)
        plt.plot(xi, yi, base_color[cls],marker="x",alpha=score)
    print(".",end="")
plt.xlim(-15,15)
plt.ylim(-15,15)
plt.show()
print("finish plotting")